In [1]:
import astropy
from astropy.io import fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib.colors import LogNorm
from astropy import units as u
import galpy
from galpy.orbit import Orbit
from astropy.coordinates import SkyCoord
from galpy.potential import MWPotential2014
from astropy.coordinates import SkyCoord, Galactocentric, CartesianDifferential
from matplotlib.colors import Normalize
from matplotlib.patches import Ellipse, Circle
import astropy.units as u
from astropy.coordinates import ICRS
import astropy.coordinates as apycoords
from astropy.coordinates import CylindricalRepresentation, CylindricalDifferential

# Error propegation for velocity components

## **Summary of Variables**
| Symbol | Description |
|--------|------------|
| \( v_phi} \) | Azimuthal velocity (km/s) |
| \( v_R \) | Radial velocity (km/s) |
| \( v_Z \) | Vertical velocity (km/s) |
| \( rho \) | Galactocentric cylindrical radius (km) |
| \( d \) | Distance to the star (pc) |
| \( mu_alpha \) | Proper motion in RA (mas/yr) |
| \( mu_delta \) | Proper motion in Dec (mas/yr) |
| \( sigma_mu_alpha \) | Uncertainty in proper motion RA (mas/yr) |
| \( sigma_mu_delta \) | Uncertainty in proper motion Dec (mas/yr) |
| \( v_los \) | Radial velocity (km/s) |
| \( sigma_v_los \) | Uncertainty in radial velocity (km/s) |


In [25]:
vphi_data = 'v_dataset.fits'

with fits.open(vphi_data) as hdul:
    data = hdul[1].data  
    
    df_v = pd.DataFrame({col.name: data[col.name].byteswap().newbyteorder() if data[col.name].dtype.byteorder == '>' else data[col.name]
                       for col in hdul[1].columns})

In [26]:
pd.set_option('display.max_columns', None)

df_v.head()

,source_id,l,b,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ruwe,radial_velocity,radial_velocity_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,catwise_w1,catwise_w2,mh_xgboost,teff_xgboost,logg_xgboost,in_training_sample,col1,col2,Source,RA_ICRS,DE_ICRS,rgeo,b_rgeo_x,B_rgeo_xa,rpgeo,b_rpgeo_x,B_rpgeo_xa,Flag,angDist,fpu,E(B-V),distance_kpc,R,Z,R_gal,phi_gal,Z_gal,v_phi,v_R,v_Z
0,15741055975040,176.739184,-48.572035,45.136038,0.335043,1.439792,0.018947,-0.711280,0.017718,-1.412098,0.016528,1.036041,-0.738894,0.316921,10.254021,10.750235,9.595748,8.152,8.198,-0.144,5065.8,2.993,True,45.136038,0.335043,15741055975040,45.136038,0.335043,695.683899,683.627625,707.396423,696.278320,688.270874,707.143982,10033,0.0,0.013160,0.104689,696.278320,8.582006,-0.522061,470.485258,0.055731,-520.632142,243.095762,-14.428048,4.529900
1,66627828480768,176.483565,-48.171322,45.305053,0.736093,0.534038,0.020692,3.309832,0.022959,1.594356,0.022822,1.349013,41.607450,1.323430,10.508830,11.150953,9.741709,7.891,7.964,-0.401,4499.0,1.916,False,45.305053,0.736093,66627828480768,45.305053,0.736093,1884.275020,1821.247560,1947.302730,1883.143550,1807.318600,1944.665770,10033,0.0,0.038747,0.093026,1883.143550,9.375831,-1.403210,1268.325930,0.060773,-1399.358389,237.364263,42.346437,-2.860027
2,82467667849472,176.209301,-48.607026,44.866246,0.561503,2.209985,0.016049,-4.869755,0.015797,-12.678339,0.013950,1.018742,-32.461674,0.205614,10.651456,11.169626,9.980610,8.496,8.558,0.114,4938.3,3.203,True,44.866246,0.561503,82467667849472,44.866246,0.561503,452.636078,448.701294,456.615540,452.602692,449.257355,455.432892,10033,0.0,0.007262,0.086075,452.602692,8.420638,-0.339539,308.483632,0.064184,-338.602406,230.329127,-51.045366,13.944756
3,101193725229056,175.755174,-48.727781,44.569524,0.689953,0.528788,0.024241,3.539184,0.027864,-1.599436,0.021891,1.176748,91.295900,4.830661,14.332739,14.816101,13.685076,12.220,12.275,-0.478,4980.2,3.319,False,44.569524,0.689953,101193725229056,44.569524,0.689953,1908.478270,1815.024170,2014.483890,1868.603880,1800.940800,1949.831300,10033,0.0,0.045842,0.078062,1868.603880,9.351665,-1.404413,1245.041147,0.073347,-1400.635394,217.472712,62.829005,-53.068842
4,130399502833792,175.789759,-48.328584,44.868872,0.955080,0.847929,0.017479,3.382907,0.017619,-0.291235,0.015982,1.087555,1.959265,0.328298,11.215295,11.810178,10.481738,8.728,8.815,-0.159,4643.6,2.398,True,44.868872,0.955080,130399502833792,44.868872,0.955080,1178.407470,1155.319820,1203.711180,1177.687260,1154.219600,1201.322750,10033,0.0,0.020614,0.077163,1177.687260,8.903067,-0.879697,793.842786,0.072479,-877.288236,231.337541,1.866465,13.969564


## 1. Uncertainty in \( v_phi \) (Azimuthal Velocity)

### **Definition**
The azimuthal velocity \( v_phi \) is given by:

$$
v_{\phi} = - \dot{\phi} \cdot \rho
$$

### **Error Propagation**
The uncertainty in \( v_{\phi} \) is:

$$
\sigma_{v_{\phi}} = \sqrt{ \left( \rho \sigma_{\dot{\phi}} \right)^2 + \left( \dot{\phi} \sigma_{\rho} \right)^2 }
$$

where:

- \( \rho \) = **Galactocentric cylindrical radius** (km)
- \( \dot{\phi} \) = **Angular velocity** (rad/s)
- \( \sigma_{\dot{\phi}} \) = **Uncertainty in angular velocity** (rad/s)
- \( \sigma_{\rho} \) = **Uncertainty in Galactocentric radius** (km)

#### **Uncertainty in \( \dot{\phi} \)**
The angular velocity \( \dot{\phi} \) is derived from the proper motions:

$$
\sigma_{\dot{\phi}} = \sqrt{ \left(\frac{\cos(\delta)}{d} \sigma_{\mu_{\alpha*}} \right)^2 + \left(\frac{\sin(\delta)}{d} \sigma_{\mu_{\delta}} \right)^2 + \left( \frac{\mu_{\alpha*} \cos(\delta) + \mu_{\delta} \sin(\delta)}{d^2} \sigma_d \right)^2 }
$$

where:
- \( \delta \) = **Declination** (degrees)
- \( d \) = **Distance to the star** (pc)
- \( \mu_{\alpha*} \) = **Proper motion in Right Ascension** (mas/yr)
- \( \mu_{\delta} \) = **Proper motion in Declination** (mas/yr)
- \( \sigma_{\mu_{\alpha*}} \), \( \sigma_{\mu_{\delta}} \) = **Uncertainties in proper motions** (mas/yr)
- \( \sigma_d \) = **Uncertainty in distance** (pc)

#### **Uncertainty in \( \rho \)**
The Galactocentric radius \( \rho \) depends on distance \( d \):

$$
\sigma_{\rho} = \left| \frac{\partial \rho}{\partial d} \right| \sigma_d
$$

where:

$$
\frac{\partial \rho}{\partial d} = \frac{d \cos(\delta) \cos(\alpha) - d_{\odot}}{\rho} \cos(\delta) \cos(\alpha) + \frac{d \cos(\delta) \sin(\alpha)}{\rho} \cos(\delta) \sin(\alpha)
$$

where:
- \( \alpha \) = **Right Ascension** (degrees)
- \( d_{\odot} \) = **Distance from the Sun to the Galactic Center** (kpc)

In [51]:
# Constants
mas_to_rad = 4.84814e-9 * u.rad / u.mas  # Convert mas to radians
yr_to_s = (1 / (3.154e7 * u.s / u.yr))  # Convert years to seconds
pc_to_km = 3.086e13 * u.km / u.pc  # Convert parsecs to kilometers

# Define the Sun's velocity with respect to the Galactic center
v_sun = CartesianDifferential([11.1, 245., 7.25] * u.km / u.s)

# Define the Galactocentric frame
gc_frame = Galactocentric(galcen_distance=8.1 * u.kpc, 
                          z_sun=25 * u.pc, 
                          galcen_v_sun=v_sun)


In [52]:
# Extract necessary data
ra = df_v['ra'].values * u.deg
dec = df_v['dec'].values * u.deg
distance = df_v['rpgeo'].values * u.pc  
pmra = df_v['pmra'].values * u.mas / u.yr
pmdec = df_v['pmdec'].values * u.mas / u.yr
vlos = df_v['radial_velocity'].values * u.km / u.s

# Extract uncertainty in distance
b_rpgeo_x_pc = df_v['b_rpgeo_x'].values * u.pc  # Lower bound
B_rpgeo_xa_pc = df_v['B_rpgeo_xa'].values * u.pc  # Upper bound

# Compute symmetric uncertainty in distance
distance_err_pc = ((B_rpgeo_xa_pc - distance) + (distance - b_rpgeo_x_pc)) / 2
distance_err_km = distance_err_pc.to(u.km)  # Convert to km
distance_km = distance.to(u.km)  # Convert distance to km

# Extract proper motion uncertainties
pmra_err = df_v['pmra_error'].values * u.mas / u.yr
pmdec_err = df_v['pmdec_error'].values * u.mas / u.yr


In [42]:
# Create a SkyCoord object for all sources
coords = ICRS(ra=ra, dec=dec, distance=distance, pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=vlos)

# Transform all coordinates to the Galactocentric frame
cg = coords.transform_to(gc_frame)
cg.representation = 'cylindrical'

# Convert to cylindrical position and velocity representations
cg_cyl = cg.represent_as(CylindricalRepresentation)
cg_cyl_vel = cg.represent_as(CylindricalRepresentation, CylindricalDifferential).differentials['s']

# Extract rho and d_phi
rho_km = cg_cyl.rho.to(u.km)  # Convert rho from pc to km
d_phi_rad_s = cg_cyl_vel.d_phi.to(u.rad / u.s)  # Convert d_phi from rad/yr to rad/s

# Compute v_phi
v_phi_kms = df_v['v_phi']


In [55]:
# Define conversion factors
mas_to_rad = 4.84814e-9 * u.rad / u.mas  # Converts mas to radians
yr_to_s = (1 / (3.154e7 * u.s / u.yr))  # Converts years to seconds

# Convert proper motion errors from mas/yr to rad/s correctly
pmra_err_rad_s = (pmra_err * mas_to_rad * yr_to_s).to(1 / u.s, equivalencies=u.dimensionless_angles())
pmdec_err_rad_s = (pmdec_err * mas_to_rad * yr_to_s).to(1 / u.s, equivalencies=u.dimensionless_angles())

# Convert proper motion values from mas/yr to rad/s
pmra_rad_s = (pmra * mas_to_rad * yr_to_s).to(1 / u.s, equivalencies=u.dimensionless_angles())
pmdec_rad_s = (pmdec * mas_to_rad * yr_to_s).to(1 / u.s, equivalencies=u.dimensionless_angles())


# Convert distance_km to a unitless quantity before using it in division
distance_km_unitless = distance_km / u.km  # Dimensionless

# Convert dec to radians before applying trigonometric functions
cos_dec = np.cos(dec.to(u.rad).value)  # Now unitless
sin_dec = np.sin(dec.to(u.rad).value)  # Now unitless

# Compute total angular velocity error (σ_{d_phi})
d_phi_err_rad_s = np.sqrt(
    ((cos_dec / distance_km_unitless) * pmra_err_rad_s)**2 +
    ((sin_dec / distance_km_unitless) * pmdec_err_rad_s)**2 +
    (((pmra_rad_s * cos_dec + pmdec_rad_s * sin_dec) / (distance_km_unitless**2)) * (distance_err_km / u.km))**2
).to(1 / u.s)

print(d_phi_err_rad_s)


[1.44696775e-34 3.26934962e-34 4.13321338e-34 ... 1.98179894e-33
 5.30714676e-34 1.39649339e-34] 1 / s


In [50]:
# Convert distance_km to a unitless quantity before using it in division
distance_km_unitless = distance_km / u.km  # Now it's dimensionless

# Compute total angular velocity error (σ_{d_phi})
d_phi_err_rad_s = np.sqrt(
    ((np.cos(dec) / distance_km_unitless) * pmra_err_rad_s)**2 +
    ((np.sin(dec) / distance_km_unitless) * pmdec_err_rad_s)**2 +
    (((pmra_rad_s * np.cos(dec) + pmdec_rad_s * np.sin(dec)) / (distance_km_unitless**2)) * (distance_err_km / u.km))**2
).to(1 / u.s)

print(d_phi_err_rad_s)

[1.44696775e-34 3.26934962e-34 4.13321338e-34 ... 1.98179894e-33
 5.30714676e-34 1.39649339e-34] 1 / s


In [47]:
# Convert cos and sin terms to unitless before using them in calculations
cos_dec = np.cos(dec.to(u.rad).value)  # Now unitless
sin_ra = np.sin(ra.to(u.rad).value)  # Now unitless
cos_ra = np.cos(ra.to(u.rad).value)  # Now unitless

# Compute uncertainty in rho (σ_ρ)
d_rho_dd = np.abs(((distance_km * cos_dec) - (8.1 * u.kpc).to(u.km)) * cos_ra / rho_km) + \
           np.abs((distance_km * cos_dec) * sin_ra / rho_km)

rho_err_km = (d_rho_dd * distance_err_km).to(u.km)

In [48]:
# Compute uncertainty in v_phi
v_phi_err_kms = np.sqrt(
    (rho_km**2 * d_phi_err_rad_s.to(1 / u.s, equivalencies=u.dimensionless_angles())**2) + 
    (d_phi_err_rad_s.to(1 / u.s, equivalencies=u.dimensionless_angles())**2 * rho_err_km**2)
).to(u.km / u.s)


# Store in DataFrame
df_v['v_phi_error'] = v_phi_err_kms.value


In [49]:
df_v['v_phi_error'].describe()

count    3.404894e+06
mean     9.795104e-17
std      1.636622e-16
min      6.721591e-19
25%      3.161996e-17
50%      6.581216e-17
75%      1.215140e-16
max      6.810075e-14
Name: v_phi_error, dtype: float64

## 2. Uncertainty in \( v_R \) (Radial Velocity)

### **Definition**
The radial velocity \( v_R \) is given by:

$$
v_R = \frac{d\rho}{dt}
$$

### **Error Propagation**
The uncertainty in \( v_R \) is:

$$
\sigma_{v_R} = \sqrt{\sigma_{d\rho/dt}^2}
$$

where:

$$
\sigma_{d\rho/dt} = \sqrt{\sigma_{\mu_{\alpha*}}^2 + \sigma_{\mu_{\delta}}^2}
$$

since the uncertainty in \( v_R \) comes from the proper motions.

In [ ]:
# Extract distance and its uncertainties
distance_pc = df_vphi['rpgeo'].values * u.pc  # Distance in parsecs
b_rpgeo_x_pc = df_vphi['b_rpgeo_x'].values * u.pc  # Lower bound
B_rpgeo_xa_pc = df_vphi['B_rpgeo_xa'].values * u.pc  # Upper bound

# Compute symmetric uncertainty in distance
distance_err_pc = ((B_rpgeo_xa_pc - distance_pc) + (distance_pc - b_rpgeo_x_pc)) / 2

# Convert to km
distance_km = distance_pc.to(u.km)
distance_err_km = distance_err_pc.to(u.km)

# Extract proper motion and errors
pmra = df_vphi['pmra'].values * u.mas / u.yr
pmdec = df_vphi['pmdec'].values * u.mas / u.yr
pmra_err = df_vphi['pmra_error'].values * u.mas / u.yr
pmdec_err = df_vphi['pmdec_error'].values * u.mas / u.yr

# Extract radial velocity and its uncertainty
vlos = df_vphi['radial_velocity'].values * u.km / u.s
vlos_err = df_vphi['radial_velocity_error'].values * u.km / u.s

# Convert proper motion uncertainties to km/s using distance
pmra_kms = (pmra * distance_km).to(u.km / u.s, equivalencies=u.dimensionless_angles())
pmdec_kms = (pmdec * distance_km).to(u.km / u.s, equivalencies=u.dimensionless_angles())

pmra_err_kms = (pmra_err * distance_km).to(u.km / u.s, equivalencies=u.dimensionless_angles())
pmdec_err_kms = (pmdec_err * distance_km).to(u.km / u.s, equivalencies=u.dimensionless_angles())

# Compute v_R and v_Z components
v_R_kms = cg_cyl_vel.d_rho.to(u.km / u.s)  # Radial velocity component
v_Z_kms = cg_cyl_vel.d_z.to(u.km / u.s)    # Vertical velocity component

# Compute uncertainty in v_R (Radial component)
v_R_err_kms = np.sqrt((pmra_err_kms**2) + (pmdec_err_kms**2))

# Compute uncertainty in v_Z (Vertical component)
v_Z_err_kms = np.sqrt((pmra_err_kms**2) + (pmdec_err_kms**2) + (vlos_err**2))

# Store in DataFrame
df_vphi['v_R_error'] = v_R_err_kms.value
df_vphi['v_Z_error'] = v_Z_err_kms.value

# Print summary statistics
print("v_R_error Summary:\n", df_vphi['v_R_error'].describe())
print("v_Z_error Summary:\n", df_vphi['v_Z_error'].describe())

# Plot histograms for v_R and v_Z uncertainties
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].hist(df_vphi['v_R_error'], bins=500, color='b', alpha=0.7)
axs[0].set_xlabel(r'$\sigma_{v_R}$ (km/s)', fontsize=12)
axs[0].set_ylabel('Number of Stars', fontsize=12)
axs[0].set_title('Distribution of $v_R$ Uncertainties', fontsize=14)
axs[0].grid(True)

axs[1].hist(df_vphi['v_Z_error'], bins=500, color='g', alpha=0.7)
axs[1].set_xlabel(r'$\sigma_{v_Z}$ (km/s)', fontsize=12)
axs[1].set_ylabel('Number of Stars', fontsize=12)
axs[1].set_title('Distribution of $v_Z$ Uncertainties', fontsize=14)
axs[1].grid(True)

plt.tight_layout()
plt.show()


---

## 3. Uncertainty in \( v_Z \) (Vertical Velocity)

### **Definition**
The vertical velocity \( v_Z \) is given by:

$$
v_Z = \frac{dz}{dt}
$$

### **Error Propagation**
The uncertainty in \( v_Z \) is:

$$
\sigma_{v_Z} = \sqrt{\sigma_{dZ/dt}^2 + \sigma_{v_{\text{los}}}^2}
$$

where:

$$
\sigma_{dZ/dt} = \sqrt{\sigma_{\mu_{\alpha*}}^2 + \sigma_{\mu_{\delta}}^2}
$$

since \( v_Z \) is affected by proper motions and the uncertainty in line-of-sight velocity \( v_{\text{los}} \).
